In [191]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import tweepy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [192]:
from twitter_keys import * 

In [193]:
def twitter_setup():
    # Authentication and access using keys
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Obtain authenticated API
    api = tweepy.API(auth,wait_on_rate_limit=True)
    return api

In [194]:
extractor=twitter_setup()

In [195]:
def get_tweets(api, query,rpp,limit=None):
        """Return a list of all tweets from the authenticated API"""
        tweets_list = []
        for tweet in tweepy.Cursor(api.search,query,rpp=100).items():
            tweets_list.append(tweet)
        return tweets_list

In [196]:
alltweets = get_tweets(extractor, query ='#msft', rpp=100)

print("Number of tweets extracted: {}.\n".format(len(alltweets)))

<h1>Reminder</h1>
<p><u>screen_name:</u> tweet.user.screen_name,<br>
    <u>tweet_id</u>:tweet.id
<u>username:</u> tweet.user.name,<br>
<u>created_at:</u> unicode(tweet.created_at),<br>
<u>tweet_text:</u> tweet.text,<br>
<u>User Location:</u> unicode(tweet.user.location),<br>
<u>Tweet Coordinates:</u> unicode(tweet.coordinates),<br>
<u>Retweet Count:</u> unicode(tweet.retweet_count),<br>
<u>Retweeted:</u> unicode(tweet.retweeted),<br>
<u>Phone Type:</u> unicode(tweet.source),<br>
<u>Favorite Count:</u> unicode(tweet.favorite_count),<br>
<u>Favorited:</u> unicode(tweet.favorited),<br>
<u>Replied:</u> unicode(tweet.in_reply_to_status_id_str)</p>

In [197]:
#DataFrame with the tweets
df_tweets=pd.DataFrame(data=[tweet.text for tweet in alltweets], columns=['Tweets'])
df_tweets['len']  = np.array([len(tweet.text) for tweet in alltweets])
df_tweets['ID']   = np.array([tweet.id for tweet in alltweets])
df_tweets['Date'] = np.array([tweet.created_at for tweet in alltweets])
df_tweets['Source'] = np.array([tweet.source for tweet in alltweets])
df_tweets['Likes']  = np.array([tweet.favorite_count for tweet in alltweets])
df_tweets['RTs']    = np.array([tweet.retweet_count for tweet in alltweets])

df_tweets.head()
  

In [145]:
#add the sentiment score of the tweet
analyzer = SentimentIntensityAnalyzer()

In [146]:
#add a column to the dataframe for the sentiment score
df_tweets['sentiment_pos']=0
df_tweets['sentiment_neg']=0
df_tweets['sentiment_neu']=0
df_tweets['sentiment_compound']=0

In [147]:
for tweet in alltweets:
    id_tweet=tweet.id
    tempTweet = str(tweet.text.encode('utf-8'))
    dict_sentiment=analyzer.polarity_scores(tempTweet)
    sentiment_pos=dict_sentiment['pos']
    sentiment_neg=dict_sentiment['neg']
    sentiment_neu=dict_sentiment['neu']
    sentiment_compound=dict_sentiment['compound']
    df_tweets.loc[df_tweets.ID==id_tweet,'sentiment_pos']=sentiment_pos
    df_tweets.loc[df_tweets.ID==id_tweet,'sentiment_neg']=sentiment_neg
    df_tweets.loc[df_tweets.ID==id_tweet,'sentiment_neu']=sentiment_neu
    df_tweets.loc[df_tweets.ID==id_tweet,'sentiment_compound']=sentiment_compound


In [198]:
df_tweets[df_tweets['sentiment_compound']>0].head(3)

In [136]:
num_pos_tweets=df_tweets[df_tweets['sentiment_compound']>0]['Tweets'].count()
num_pos_tweets

205

In [152]:
#Percentages
num_rows=df_tweets.shape[0]
perc_pos_tweets=round(100*df_tweets[df_tweets['sentiment_compound']>0]['Tweets'].count()/num_rows,1)
perc_neg_tweets=round(100*df_tweets[df_tweets['sentiment_compound']<0]['Tweets'].count()/num_rows,1)
perc_neu_tweets=round(100*df_tweets[df_tweets['sentiment_compound']==0]['Tweets'].count()/num_rows,1)
print("Percentage of neutral tweets: {}%".format(perc_neu_tweets))
print("Percentage of positive tweets: {}%".format(perc_pos_tweets))
print("Percentage of negative tweets: {}%".format(perc_neg_tweets))

Percentage of neutral tweets: 84.8%
Percentage of positive tweets: 5.0%
Percentage of negative tweets: 10.2%


In [ ]:
# Create time serie based on compound sentiment
ts_compound=pd.Series(data=df_tweets['sentiment_compound'].values, index=df_tweets['Date'])

In [ ]:
#Plot compound sentiment
ts_compound.plot()